# CHAPTER 7 - VQE: Variational Quantum Eigensolver - Qiskit Code

*Note*: You may skip the following three cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.

In [ ]:
pip install qiskit==0.39.2

In [ ]:
pip install qiskit_nature==0.4.5

In [ ]:
pip install pyscf==2.1.1

In [ ]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem

mol = Molecule(geometry=[['H', [0., 0., -0.37]],
                        ['H', [0., 0., 0.37]]])
driver = ElectronicStructureMoleculeDriver(mol, basis='sto3g', 
        driver_type=ElectronicStructureDriverType.PYSCF)
problem = ElectronicStructureProblem(driver)
secqop = problem.second_q_ops()
print(secqop[0])

In [ ]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

qconverter = QubitConverter(JordanWignerMapper())
qhamiltonian = qconverter.convert(secqop[0])
print("Qubit Hamiltonian")
print(qhamiltonian)

In [ ]:
pip install pylatexenc==2.10

In [ ]:
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement="linear", 
    insert_barriers = True)
ansatz.decompose().draw("mpl")

In [ ]:
from qiskit.algorithms import VQE
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals
import numpy as np
from qiskit.algorithms.optimizers import COBYLA

seed = 1234
np.random.seed(seed)
algorithm_globals.random_seed = seed

optimizer = COBYLA()

initial_point = np.random.random(ansatz.num_parameters)
quantum_instance = QuantumInstance(backend = 
    Aer.get_backend('aer_simulator_statevector'))

vqe = VQE(ansatz=ansatz, optimizer=optimizer, 
    initial_point=initial_point, 
    quantum_instance=quantum_instance)

In [ ]:
result = vqe.compute_minimum_eigenvalue(qhamiltonian)
print(result)

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
solver = NumPyMinimumEigensolver()
result = solver.compute_minimum_eigenvalue(qhamiltonian)
print(result)

In [ ]:
from qiskit.algorithms import VQD
vqd = VQD(ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point,
    quantum_instance=quantum_instance,
    k = 2)
result = vqd.compute_eigenvalues(qhamiltonian)
print(result)

In [ ]:
from qiskit_nature.algorithms import GroundStateEigensolver

solver = GroundStateEigensolver(qconverter, vqe)
result = solver.solve(problem)
print(result)

In [ ]:
from qiskit_nature.algorithms import VQEUCCFactory

vqeuccf = VQEUCCFactory(quantum_instance = quantum_instance)

In [ ]:
vqeuccf.get_solver(problem, qconverter).ansatz.decompose().draw("mpl")


In [ ]:
solver = GroundStateEigensolver(qconverter, vqeuccf)
result = solver.solve(problem)
print(result)

*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [ ]:
ibm_token="1234"



In [ ]:
from qiskit.providers.aer import AerSimulator
from qiskit import IBMQ

IBMQ.save_account(ibm_token)
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_manila')
quantum_instance = QuantumInstance(
    backend = AerSimulator.from_backend(backend),
    seed_simulator=seed, seed_transpiler = seed, shots = 1024)

In [ ]:
vqe = VQE(
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point,
    quantum_instance=quantum_instance
)

solver = GroundStateEigensolver(qconverter, vqe)
result = solver.solve(problem)
print(result)

In [ ]:
from qiskit.utils.mitigation import CompleteMeasFitter

quantum_instance = QuantumInstance(
    backend = AerSimulator.from_backend(backend),
    measurement_error_mitigation_cls=CompleteMeasFitter,
    seed_simulator=seed, seed_transpiler = seed, shots = 1024)

In [ ]:
vqe = VQE(
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point,
    quantum_instance=quantum_instance
)

solver = GroundStateEigensolver(qconverter, vqe)
result = solver.solve(problem)
print(result)

In [ ]:
from qiskit_nature.runtime import VQEClient
backend = provider.get_backend('ibmq_manila')

vqe = VQEClient(
    ansatz=ansatz,
    provider=provider,
    backend=backend,
    shots=1024,
    initial_point = initial_point,
    measurement_error_mitigation=False
)  

solver = GroundStateEigensolver(qconverter, vqe)
result = solver.solve(problem)
print(result)

In [ ]:
from qiskit_nature.runtime import VQEClient
backend = provider.get_backend('ibmq_manila')

vqe = VQEClient(
    ansatz=ansatz,
    provider=provider,
    backend=backend,
    shots=1024,
    initial_point = initial_point,
    measurement_error_mitigation=True
)  

solver = GroundStateEigensolver(qconverter, vqe)
result = solver.solve(problem)
print(result)

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator

estimator= Estimator()

vqe = VQE(
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point,
    estimator=estimator
)

result = vqe.compute_minimum_eigenvalue(qhamiltonian)

print(result)